In [1]:
%%capture
!pip install sentence-transformers

In [3]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

#Our sentences we like to encode
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.',
    'The quick brown fox jumps over the lazy dog.']

#Sentences are encoded by calling model.encode()
embeddings = model.encode(sentences)

In [4]:
#Print the embeddings
for sentence, embedding in zip(sentences, embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding.shape)
    print("")

Sentence: This framework generates embeddings for each input sentence
Embedding: (384,)

Sentence: Sentences are passed as a list of string.
Embedding: (384,)

Sentence: The quick brown fox jumps over the lazy dog.
Embedding: (384,)



In [6]:
from sentence_transformers import util

# Two lists of sentences
sentences1 = ['The cat sits outside',
             'A man is playing guitar',
             'The new movie is awesome']

sentences2 = ['The dog plays in the garden',
              'A woman watches TV',
              'The new movie is so great']

#Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)

#Compute cosine-similarities
cosine_scores = util.cos_sim(embeddings1, embeddings2)

#Output the pairs with their score
for index1, s1 in enumerate(sentences1):
  for index2, s2 in enumerate(sentences2):
    print("{} \t\t {} \t\t Score: {:.4f}".format(s1, s2, cosine_scores[index1][index2]))

The cat sits outside 		 The dog plays in the garden 		 Score: 0.2838
The cat sits outside 		 A woman watches TV 		 Score: 0.1310
The cat sits outside 		 The new movie is so great 		 Score: -0.0029
A man is playing guitar 		 The dog plays in the garden 		 Score: 0.2277
A man is playing guitar 		 A woman watches TV 		 Score: -0.0327
A man is playing guitar 		 The new movie is so great 		 Score: -0.0136
The new movie is awesome 		 The dog plays in the garden 		 Score: 0.0543
The new movie is awesome 		 A woman watches TV 		 Score: -0.0502
The new movie is awesome 		 The new movie is so great 		 Score: 0.8939


In [10]:
# Single list of sentences - Possible tens of thousands of sentences
sentences = ['The cat sits outside',
             'A man is playing guitar',
             'I love pasta',
             'The new movie is awesome',
             'The cat plays in the garden',
             'A woman watches TV',
             'The new movie is so great',
             'Do you like pizza?']

paraphrases = util.paraphrase_mining(model, sentences)

print(len(paraphrases))

28


In [11]:
for paraphrase in paraphrases:
    score, i, j = paraphrase
    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences[i], sentences[j], score))

The new movie is awesome 		 The new movie is so great 		 Score: 0.8939
The cat sits outside 		 The cat plays in the garden 		 Score: 0.6788
I love pasta 		 Do you like pizza? 		 Score: 0.5096
I love pasta 		 The new movie is so great 		 Score: 0.2560
I love pasta 		 The new movie is awesome 		 Score: 0.2440
A man is playing guitar 		 The cat plays in the garden 		 Score: 0.2105
The new movie is awesome 		 Do you like pizza? 		 Score: 0.1969
The new movie is so great 		 Do you like pizza? 		 Score: 0.1692
The cat sits outside 		 A woman watches TV 		 Score: 0.1310
The cat plays in the garden 		 Do you like pizza? 		 Score: 0.0900
The cat plays in the garden 		 A woman watches TV 		 Score: 0.0629
The cat plays in the garden 		 The new movie is so great 		 Score: 0.0591
A woman watches TV 		 Do you like pizza? 		 Score: 0.0417
The cat sits outside 		 A man is playing guitar 		 Score: 0.0363
I love pasta 		 A woman watches TV 		 Score: 0.0359
The new movie is awesome 		 The cat plays in th

In [14]:
import torch

# Corpus with example sentences
corpus = [
    "A man is eating food.",
    "A man is eating a piece of bread.",
    "The girl is carrying a baby.",
    "A man is riding a horse.",
    "A woman is playing violin.",
    "Two men pushed carts through the woods.",
    "A man is riding a white horse on an enclosed ground.",
    "A monkey is playing drums.",
    "A cheetah is running behind its prey.",
]
corpus_embeddings = model.encode(corpus, convert_to_tensor=True)

# Query sentences:
queries = [
    "A man is eating pasta.",
    "Someone in a gorilla costume is playing a set of drums.",
    "A cheetah chases prey on across a field.",
]


# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = min(5, len(corpus))
for query in queries:
    query_embedding = model.encode(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")


    print("Query:", query)
    print(top_results)

    print("\nTop 5 most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print(corpus[idx], "(Score: {:.4f})".format(score))





Query: A man is eating pasta.
torch.return_types.topk(
values=tensor([0.7035, 0.5272, 0.1889, 0.1047, 0.0980]),
indices=tensor([0, 1, 3, 6, 8]))

Top 5 most similar sentences in corpus:
A man is eating food. (Score: 0.7035)
A man is eating a piece of bread. (Score: 0.5272)
A man is riding a horse. (Score: 0.1889)
A man is riding a white horse on an enclosed ground. (Score: 0.1047)
A cheetah is running behind its prey. (Score: 0.0980)




Query: Someone in a gorilla costume is playing a set of drums.
torch.return_types.topk(
values=tensor([0.6433, 0.2564, 0.1389, 0.1191, 0.1080]),
indices=tensor([7, 4, 3, 6, 8]))

Top 5 most similar sentences in corpus:
A monkey is playing drums. (Score: 0.6433)
A woman is playing violin. (Score: 0.2564)
A man is riding a horse. (Score: 0.1389)
A man is riding a white horse on an enclosed ground. (Score: 0.1191)
A cheetah is running behind its prey. (Score: 0.1080)




Query: A cheetah chases prey on across a field.
torch.return_types.topk(
values=t